# Selecting the top 3 Industries

In [1]:
import os
import sys  
from pathlib import Path
curr_path = str(Path(os.getcwd()).parent)
sys.path.append(curr_path)
import seaborn as sns

from scripts.constants import *
from scripts.plotting import *
from scripts.misc_changes import *

from pyspark.sql.functions import count, when, col
from pyspark.sql import Row
import matplotlib.pyplot as plt
import numpy as np

In [2]:
spark = create_spark()

your 131072x1 screen size is bogus. expect trouble
23/10/13 13:21:50 WARN Utils: Your hostname, DESKTOP-IUQJ8D5 resolves to a loopback address: 127.0.1.1; using 172.23.100.88 instead (on interface eth0)
23/10/13 13:21:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/13 13:21:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/13 13:21:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/10/13 13:21:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/10/13 13:21:52 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
# reading all the relevant files as pandas dataframe
PREFIX = "."

average_and_median_dollar_per_tag = pd.read_csv(PREFIX + CURATED_AVERAGE_AND_MEDIAN_DOLLAR_PER_TAG_PATH)
average_and_median_fraud_prob = pd.read_csv(PREFIX + CURATED_AVERAGE_AND_MEDIAN_FRAUD_PROB_PATH)
average_earning = pd.read_csv(PREFIX + CURATED_AVERAGE_EARNING_PATH)
number_of_transactions = pd.read_csv(PREFIX + CURATED_NUMBER_OF_TRANSACTIONS_PATH)
revenue_a = pd.read_csv(PREFIX + REVENUE_A_PATH)
revenue_b = pd.read_csv(PREFIX + REVENUE_B_PATH)
revenue_c = pd.read_csv(PREFIX + REVENUE_C_PATH)
revenue_d = pd.read_csv(PREFIX + REVENUE_D_PATH)
revenue_e = pd.read_csv(PREFIX + REVENUE_E_PATH)
takerate_mean_median = pd.read_csv(PREFIX + CURATED_TAKERATE_PATH)


In [4]:
# sorting the dataframes by Industry

average_and_median_dollar_per_tag = average_and_median_dollar_per_tag.sort_values(by=MAPPED_INDUSTRY)
average_and_median_fraud_prob = average_and_median_fraud_prob.sort_values(by=MAPPED_INDUSTRY)
average_earning = average_earning.sort_values(by=MAPPED_INDUSTRY)
number_of_transactions = number_of_transactions.sort_values(by=MAPPED_INDUSTRY)
revenue_a = revenue_a.sort_values(by=MAPPED_INDUSTRY)
revenue_b = revenue_b.sort_values(by=MAPPED_INDUSTRY)
revenue_c = revenue_c.sort_values(by=MAPPED_INDUSTRY)
revenue_d = revenue_d.sort_values(by=MAPPED_INDUSTRY)
revenue_e = revenue_e.sort_values(by=MAPPED_INDUSTRY)
takerate_mean_median = takerate_mean_median.sort_values(by=MAPPED_INDUSTRY)

Combining all the relevant dataframe to one dataframe based on the common column Industry. Filling any empty values with 0 and removing repeated columns from all the dataframes. 

In [5]:
dataframes = [average_and_median_dollar_per_tag,
average_and_median_fraud_prob,
number_of_transactions,
revenue_a,
revenue_b,
revenue_c,
revenue_d,
revenue_e,
takerate_mean_median]

common_column = MAPPED_INDUSTRY  

final_df = dataframes[0]

for df in dataframes[1:]:
    final_df = final_df.merge(df, on=common_column, how=OUTER_JOIN)

final_df = final_df.fillna(0)


In [6]:
repetitive_clm = ['industry_tags_y', 'industry_tags_x']
final_df = final_df.drop(columns=repetitive_clm)

In [7]:
final_df

,MappedIndustry,dollar_value_avg,dollar_value_median,fraud_probability_avg,fraud_probability_median,industry_tags,number_of_transactions,Revenue_a_Count,Revenue_b_Count,Revenue_c_Count,Revenue_d_Count,Revenue_e_Count,Mean_Takerate,Median_Takerate
0,A,9396.485,7759.340,34.751,33.820,jewelry watch clock silverware,3749,8,42,37,4,0,3.451099,3.450
1,B,200.001,131.040,31.688,31.344,shoe,331688,85,59,37,0,4,4.592973,4.770
2,C,191.744,155.170,29.133,29.133,computer programming data processing integrate...,397913,78,54,47,8,4,4.294398,4.420
3,D,459.566,371.155,28.881,29.520,stationery office printing writing paper,131088,56,47,43,11,4,3.963913,3.790
4,E,255.977,64.540,28.897,28.690,artist supply craft,362069,84,48,54,5,2,4.366373,4.470
5,F,64.205,51.830,0.000,0.000,digital goods books movies music,984798,95,53,39,5,3,4.645385,5.050
6,G,251.571,178.500,62.993,62.993,books periodicals newspapers,302505,63,61,34,3,3,4.399817,4.400
7,H,360.641,234.960,29.555,29.555,motor vehicle new parts,235662,53,60,30,2,6,4.323444,4.330
8,I,88.144,40.840,28.568,28.361,opticians optical goods eyeglasses,647307,54,56,35,5,1,4.376755,4.390
9,J,166.911,63.480,30.338,30.024,computers computer peripheral equipment software,551214,78,64,34,3,2,4.585856,4.620


Since the fraud probability median and average gives similar value, we can consider it as normal distribution and consider just fraud_probability_avg. For the ranking metrix that we use, we are 

In [8]:
final_df['fraud_probability_avg'] = 100 - final_df['fraud_probability_avg']


Taking the relevent columns for the ranking of the industries to find the top 3 industries

In [15]:
scoring_column = ['dollar_value_avg', 'dollar_value_median',
       'fraud_probability_avg',
       'number_of_transactions', 'Revenue_a_Count', 'Revenue_b_Count',
       'Revenue_c_Count', 'Revenue_d_Count', 'Revenue_e_Count',
       'Mean_Takerate', 'Median_Takerate']

In [16]:
# assuming equal weight

weight = 1/len(scoring_column)

In [17]:
# for each industry (every row in the final_df dataframe has information about an unique Industry) 
# we calculate the score for ranking

def ranking(row: DataFrame) -> float:
    """
    Calculting the score for each merchant
    - Parameters: 
        - row: dataframe with one row (information about one merchant)
    - Returns:
        - Score for each industry
    """
    score = 0
    # function to score each industry
    for x in scoring_column:
        score+= (weight*row[x])
    
    return score

In [18]:
rank = {}

In [19]:
# saving the score of each industry to a dictionary

for index, row in final_df.iterrows():
    score = ranking(row)
    rank[row[MAPPED_INDUSTRY]] = score

Convering the dictionary to a pandas dataframe, sorting to find the top 3 industries and saving it to a csv file

In [20]:
sorted_rank = dict(sorted(rank.items(), key=lambda item: item[1], reverse=True))
sorted_rank_df = pd.DataFrame({MAPPED_INDUSTRY:  list(sorted_rank.keys()), 'Score': list(sorted_rank.values())})
print(sorted_rank_df)
sorted_rank_df.to_csv(PREFIX + SORTED_INDUSTRY_PATH)

   MappedIndustry          Score
0               X  126594.929543
1               P  114358.044288
2               F   89565.339126
3               V   71958.372096
4               K   66762.937325
5               U   61457.163333
6               I   58878.834796
7               J   50154.932623
8               C   36230.045036
9               T   34781.415863
10              E   32969.314216
11              B   30207.428725
12              G   27558.625256
13              W   24291.127310
14              M   22229.357675
15              H   21498.881768
16              N   17207.167675
17              O   16448.450866
18              Y   15716.026230
19              R   12511.659638
20              D   12014.417628
21              S    2744.672326
22              Q    2678.561495
23              A    1915.270464
24              L    1817.639544
